In [1]:
import psycopg2
import clickhouse_connect
import pandas as pd
import os
import sys


# Подключение к PostgreSQL
def conn_pg():
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="dbpg",
            user="user",
            password="password",
            port=5432
        )
        # print("Connected to PostgreSQL")
        # cur = conn.cursor()
        # cur.execute(query)
        # output = cur.fetchall()
        # print("PostgreSQL version:", db_version)
        # if output:
        #     return
        # else:
        #     print("No output from the query")
        return conn    
    except Exception as e:
        print(f"Error connecting to PostgreSQL: {e}")
    
def conn_pg_close(conn):
    if conn:
        cur = conn.cursor()
        cur.close()
        conn.close()
        # print("PostgreSQL connection closed")


# Подключение к ClickHouse
client = clickhouse_connect.get_client(
            host='localhost'
            ,username='user'
            ,password='password'
            ,database='dbch'
            ,port=8123
            )
        # print("Connected to ClickHouse")
        # version = client.command('SELECT version()')

        # print("ClickHouse version:", version)


# import tools
# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # for subfolder

# find tools in parent dir
if os.path.isdir(os.path.join(parent_dir, 'tools')):
    # Add parent directory to sys.path if found
    sys.path.append(parent_dir)
    
else:
    # for run in spark
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../airflow/airflow_data"))
    
    # Add parent directory to sys.path
    sys.path.append(parent_dir)


from tools import pd_tools
from tools.paths import Paths
from tools.db_tools import DbTools


# import paths
root_path = '..' # for local folder
paths = Paths(root_path)
data_path = paths.data_path
tmp_path = paths.tmp_path
prod_data_path = paths.prod_data_path
dev_data_path = paths.dev_data_path
prod_db = paths.prod_db
dev_db = paths.dev_db
tmp_db = paths.tmp_db
tmp_data_path = paths.tmp_data_path

db_tools = DbTools(data_path, tmp_path, client)

In [2]:
raw_data = pd.read_csv(f'{data_path}/data-9-structure-4.csv')
raw_data.head()

C:\Users\pmz\AppData\Local\Temp\ipykernel_19620\3635358273.py:1: DtypeWarning: Columns (2,12,13,14,17,19,20,32,35,36,37,40,44) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(f'{data_path}/data-9-structure-4.csv')


,Название фильма,Hаименование на иностранном языке,Номер удостоверения,Дата регистрации удостоверения,Дата начала показа фильма,Примечание,Идентификатор записи реестра,Аннотация,Вид Фильма,Категория,...,Код,Права проката переданы,Порядковый номер переоформления/продления,Должность подписанта,ФИО подписанта,Запись удалена,Дата удаления записи,Не показывать на сайте mkrf.ru,дата создания объекта,дата обновления объекта
0,Рок’89,NaN,211004124,2024-01-30T12:00:00.000Z,NaN,NaN,10003030,NaN,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T15:41:59+03:00,2024-02-05T14:17:27+03:00
1,Мама будет против,NaN,211004024,2024-01-30T12:00:00.000Z,2024-02-01T12:00:00.000Z,NaN,10003032,Комедия,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T15:42:13+03:00,2024-02-05T18:23:14+03:00
2,ДЕВЯТЬ С ПОЛОВИНОЙ НЕДЕЛЬ,9 1/2 WEEKS,221005424,2024-01-30T12:00:00.000Z,2024-02-02T12:00:00.000Z,NaN,10003203,NaN,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T16:08:51+03:00,2024-02-06T12:12:04+03:00
3,Как я встретил её маму,NaN,111001324,2024-01-30T12:00:00.000Z,NaN,NaN,10003265,NaN,Художественный,Кино,...,"[{""rightsCategoryUse"":{""code"":""01"",""name"":""Пок...",NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T16:20:44+03:00,2024-03-26T14:41:20+03:00
4,Миссия в Москве,Mo Si Ke xing dong,121002924,2024-01-30T12:00:00.000Z,2024-02-08T12:00:00.000Z,NaN,10003499,NaN,Художественный,Кино,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T18:10:00+03:00,2024-01-30T18:10:00+03:00


In [3]:
pd_tools.df_info(raw_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103848 entries, 0 to 103847
Data columns (total 48 columns):
 #   Column                                                 Non-Null Count   Dtype  
---  ------                                                 --------------   -----  
 0   Название фильма                                        103848 non-null  object 
 1   Hаименование на иностранном языке                      51894 non-null   object 
 2   Номер удостоверения                                    103848 non-null  object 
 3   Дата регистрации удостоверения                         103846 non-null  object 
 4   Дата начала показа фильма                              99996 non-null   object 
 5   Примечание                                             29239 non-null   object 
 6   Идентификатор записи реестра                           103848 non-null  int64  
 7   Аннотация                                              92858 non-null   object 
 8   Вид Фильма                        

'First 5 rows in df \n'

,Название фильма,Hаименование на иностранном языке,Номер удостоверения,Дата регистрации удостоверения,Дата начала показа фильма,Примечание,Идентификатор записи реестра,Аннотация,Вид Фильма,Категория,...,Код,Права проката переданы,Порядковый номер переоформления/продления,Должность подписанта,ФИО подписанта,Запись удалена,Дата удаления записи,Не показывать на сайте mkrf.ru,дата создания объекта,дата обновления объекта
0,Рок’89,NaN,211004124,2024-01-30T12:00:00.000Z,NaN,NaN,10003030,NaN,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T15:41:59+03:00,2024-02-05T14:17:27+03:00
1,Мама будет против,NaN,211004024,2024-01-30T12:00:00.000Z,2024-02-01T12:00:00.000Z,NaN,10003032,Комедия,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T15:42:13+03:00,2024-02-05T18:23:14+03:00
2,ДЕВЯТЬ С ПОЛОВИНОЙ НЕДЕЛЬ,9 1/2 WEEKS,221005424,2024-01-30T12:00:00.000Z,2024-02-02T12:00:00.000Z,NaN,10003203,NaN,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T16:08:51+03:00,2024-02-06T12:12:04+03:00
3,Как я встретил её маму,NaN,111001324,2024-01-30T12:00:00.000Z,NaN,NaN,10003265,NaN,Художественный,Кино,...,"[{""rightsCategoryUse"":{""code"":""01"",""name"":""Пок...",NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T16:20:44+03:00,2024-03-26T14:41:20+03:00
4,Миссия в Москве,Mo Si Ke xing dong,121002924,2024-01-30T12:00:00.000Z,2024-02-08T12:00:00.000Z,NaN,10003499,NaN,Художественный,Кино,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T18:10:00+03:00,2024-01-30T18:10:00+03:00


,duplicates_full
duplicates_full,15


,Название фильма,Hаименование на иностранном языке,Номер удостоверения,Дата регистрации удостоверения,Дата начала показа фильма,Примечание,Идентификатор записи реестра,Аннотация,Вид Фильма,Категория,...,Код,Права проката переданы,Порядковый номер переоформления/продления,Должность подписанта,ФИО подписанта,Запись удалена,Дата удаления записи,Не показывать на сайте mkrf.ru,дата создания объекта,дата обновления объекта
duplicates_by_cols,17600,69561,98,97113,96456,96731,15,50932,103832,103843,...,102400,103471,103839,103829,103817,103847,103846,103847,84229,80373


...

,Hаименование на иностранном языке,Дата регистрации удостоверения,Дата начала показа фильма,Примечание,Аннотация,Вид Фильма,Язык оригинала,Формат кадра,Количество серий,Метраж (для фильмов на кинопленке),...,Возрастная категория.1,Дубляж (где дублирован),Срок действия прав проката до,Права проката переданы,Порядковый номер переоформления/продления,Должность подписанта,ФИО подписанта,Запись удалена,Дата удаления записи,Не показывать на сайте mkrf.ru
nans,51954,2,3852,74609,10990,5,103844,89606,57464,91163,...,2,103193,103777,103327,103830,83849,83850,103848,103847,103848


""
nones


""
NA placeholder


""
null placeholder


""
N/A placeholder


In [13]:
raw_data.columns

Index(['Название фильма', 'Hаименование на иностранном языке',
       'Номер удостоверения', 'Дата регистрации удостоверения',
       'Дата начала показа фильма', 'Примечание',
       'Идентификатор записи реестра', 'Аннотация', 'Вид Фильма', 'Категория',
       'Язык оригинала', 'Формат кадра', 'Количество серий',
       'Метраж (для фильмов на кинопленке)',
       'Количество частей/рулонов (для фильмов на кинопленке)', 'Цвет',
       'Продолжительность демонстрации, часы',
       'Продолжительность демонстрации, минуты', 'Формат носителя',
       'Объем носителя', 'Единица измерения', 'Студия-производитель',
       'Страна производства', 'Год производства', 'Режиссер', 'Художник',
       'Сценарист', 'Оператор', 'Композитор', 'Продюсер',
       'Права на использование фильма',
       'Категория прав на использование фильма',
       'Срок действия прав на использование фильма',
       'Срок действия прав на способ использования фильма',
       'Возрастная категория', 'Возрастная кате

In [4]:
# write duplicated values in columns
d_cols = {}

for col in raw_data.columns:
    dsum = raw_data[col].duplicated().sum()
    if dsum > 0:
        d_cols[col] = dsum
        
d_cols = pd.DataFrame.from_dict(d_cols, orient='index', columns=['duplicated_count'])
d_cols.head()


,duplicated_count
Название фильма,17600
Hаименование на иностранном языке,69561
Номер удостоверения,98
Дата регистрации удостоверения,97113
Дата начала показа фильма,96456


In [6]:
# Создание курсора
conn = conn_pg()

cur = conn.cursor()

# Создание таблицы
cur.execute('''
CREATE TABLE IF NOT EXISTS employees (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    position VARCHAR(100),
    salary NUMERIC
);
''')
print("Table 'employees' created")

# Вставка данных в таблицу
cur.execute('''
INSERT INTO employees (name, position, salary)
VALUES ('Alice', 'Manager', 70000),
        ('Bob', 'Developer', 60000),
        ('Charlie', 'Designer', 50000);
''')
print("Data inserted into 'employees' table")

# Сохранение изменений
conn.commit()


Table 'employees' created
Data inserted into 'employees' table


In [7]:
# Закрытие курсора и соединения
conn_pg_close(conn)

In [9]:
try:
    # Создание таблицы
    client.command('''
    CREATE TABLE IF NOT EXISTS dbch.employees (
        id UInt32,
        name String,
        position String,
        salary Float32
    ) ENGINE = MergeTree()
    ORDER BY id;
    ''')
    print("Table 'employees' created in ClickHouse")

except Exception as e:
    print(f"Error connecting to ClickHouse: {e}")


Table 'employees' created in ClickHouse


In [15]:
client.query('select * from dbch.employees')

In [16]:
# # Correct way to insert rows
# data = [
#     (1, 'Alice', 'Manager', 70000.0),
#     (2, 'Bob', 'Developer', 60000.0),
#     (3, 'Charlie', 'Designer', 50000.0)
# ]

# client.insert('dbch.employees', data, column_names=['id', 'name', 'position', 'salary'])